# Soft Binary Neural Network with Recurrent Forward Pass Layers for Crossbars

## Imports and Dataset

In [1]:
import numpy as np
import torch
import cv2 as cv
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from torchinfo import summary
import ast
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
def plot_history(history, num_epochs, element):
    epochs = range(len(history[list(history.keys())[0]]))
    
    fig, ax1 = plt.subplots(figsize=(12, 8))
    
    ax1.plot(epochs, history["train_loss"], label="Train Loss", color="blue")
    ax1.plot(epochs, history["val_loss"], label="Validation Loss", color="red")
    ax1.set_xlabel("Epochs", fontsize=14)
    ax1.set_ylabel("Loss", fontsize=14, color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")
    ax1.legend(loc="upper left")
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.plot(epochs, history["train_accuracy"], label="Train Accuracy", color="green")
    ax2.plot(epochs, history["val_accuracy"], label="Validation Accuracy", color="orange")
    ax2.set_ylabel("Accuracy (%)", fontsize=14, color="green")
    ax2.tick_params(axis="y", labelcolor="green")
    ax2.legend(loc="upper right")

    plt.title(f"Training and Validation Metrics for {element}", fontsize=16)
    plt.tight_layout()
    plt.show()

In [3]:
def test(model, test_loader, class_names=None):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / total_samples
    accuracy = (total_correct / total_samples) * 100

    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.2f}%")
    
    cm = confusion_matrix(all_labels, all_predictions)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    return cm

### MNIST Handwritten Digits

In [4]:
class BinarizeAndAddNoiseTransform:
    def __init__(self, threshold=0.5, noise_std=0.1):
        self.threshold = threshold
        self.noise_std = noise_std

    def __call__(self, img):
        img = transforms.ToTensor()(img)
        img = (img > self.threshold).float()
        noise = torch.randn(img.size()) * self.noise_std
        noisy_img = img + noise
        return noisy_img

binary_noise_transform = transforms.Compose([
    BinarizeAndAddNoiseTransform(threshold=0.5, noise_std=0.2)
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=binary_noise_transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=5000, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=5000, shuffle=False)

## Custom Neural Network

In [5]:
from torch.cuda.amp import GradScaler, autocast

torch.backends.cudnn.benchmark = True

class SoftBinaryRecurrentForwardNetwork(nn.Module):
    def __init__(self, G_ON, G_OFF, V_INV, R_INV, V_1, V_0, zeta, initial_factor,
                 crossbar=(8, 8), input_size=16, recurrence_size=4, output_size=4, int_norm=True,
                 hidden_layers=2, monitor_volts=False, monitor_grad=True, dropout=0.2, int_lr=0.01, device="cpu"):
        super(SoftBinaryRecurrentForwardNetwork, self).__init__()

        self.device = device

        # Crossbar Weights
        self.w = nn.Parameter(torch.empty(crossbar, device=self.device))
        nn.init.xavier_uniform_(self.w)
        self.w.data = initial_factor * self.w.to(self.device)

        # Circuit Parameters
        self.G_ON = G_ON
        self.G_OFF = G_OFF
        self.V_INV = V_INV
        self.R_INV = R_INV
        self.V_1 = V_1
        self.V_0 = V_0

        # Architecture Parameters
        self.feedback = recurrence_size
        self.data_in = crossbar[0] - recurrence_size
        self.output_size = output_size
        self.crossbar_in = crossbar[0]
        self.crossbar_out = crossbar[1]
        self.h_layers = hidden_layers
        self.r_passes = input_size // self.data_in

        # Training Variables
        self.zeta = zeta
        self.monitor_volts = monitor_volts
        self.monitor_grad = monitor_grad
        self.dropout = nn.Dropout(p=dropout)
        self.int_lr = int_lr
        self.int_norm = int_norm

    def INV_AMP(self, x):
        return -self.V_INV * torch.tanh(self.R_INV * x / self.V_INV)

    def SOFT_BIN(self, x):
        return ((self.G_ON - self.G_OFF) * torch.sigmoid(x * self.zeta) + self.G_OFF)

    def PREPROCESS(self, img):
        return (self.V_1 - self.V_0) * img.view(img.size(0), -1) + self.V_0

    def forward(self, img):
        img = self.PREPROCESS(img).to(self.device)

        g = self.SOFT_BIN(self.w).to(self.device)

        feedback = self.PREPROCESS(torch.zeros((img.shape[0], self.feedback), device=self.device))

        for r_pass in range(self.r_passes):
            x = torch.cat((feedback, img[:, r_pass * self.data_in:(r_pass + 1) * self.data_in]), dim=1)
            x = F.linear(x, g, bias=None)
            x = self.INV_AMP(x)
            feedback = x[:, -self.feedback:]

        x = self.dropout(x)

        for h_pass in range(self.h_layers):
            x = F.linear(x, g, bias=None)
            x = self.INV_AMP(x)
            x = self.dropout(x)

        return x[:, :self.output_size]

In [6]:
def tensor_stats(tensor, name="Tensor"):
    mean_magnitude = tensor.abs().mean().item()
    print(f"{name} - Mean Magnitude: {mean_magnitude:.2e}, Max: {tensor.max().item():.2e}, Min: {tensor.min().item():.2e}")
    
class SoftBinaryRecurrentForwardNetwork(nn.Module):
    def __init__(self, G_ON, G_OFF, V_INV, R_INV, V_1, V_0, zeta, initial_factor, 
                 crossbar=(8,8), input_size=16, recurrence_size=4, output_size=4, int_norm = True,
                 hidden_layers=2, monitor_volts=False, monitor_grad=True, dropout=0.2, int_lr = 0.01):
        super(SoftBinaryRecurrentForwardNetwork, self).__init__()

        # Crossbar Weights
        self.w = nn.Parameter(torch.empty(crossbar))
        nn.init.xavier_uniform_(self.w)
        self.w.data = initial_factor * self.w
        
        # Circuit Parameters
        self.G_ON = G_ON
        self.G_OFF = G_OFF
        self.V_INV = V_INV
        self.R_INV = R_INV
        self.V_1 = V_1
        self.V_0 = V_0

        # Architecture Parameters
        self.feedback = recurrence_size
        self.data_in = crossbar[0] - recurrence_size
        self.output_size = output_size
        self.crossbar_in = crossbar[0]
        self.crossbar_out = crossbar[1]
        self.h_layers = hidden_layers
        self.r_passes = input_size // self.data_in
        
        # Training Variables
        self.zeta = zeta
        self.monitor_volts = monitor_volts
        self.monitor_grad = monitor_grad
        self.dropout = nn.Dropout(p=dropout)
        self.int_lr = int_lr
        self.int_norm = int_norm

    def INV_AMP(self, x):
        return -self.V_INV * torch.tanh(self.R_INV * x / self.V_INV)

    def SOFT_BIN(self, x):
        return ((self.G_ON - self.G_OFF) * torch.sigmoid(x * self.zeta) + self.G_OFF)

    def PREPROCESS(self, img):
        return (self.V_1 - self.V_0) * img.view(img.size(0), -1) + self.V_0

    def forward(self, img):
        # Preprocessing: Two States of input (V_ON and V_OFF)
        img = self.PREPROCESS(img)

        # RRAM Soft Binarization
        g = self.SOFT_BIN(self.w).to(img.device)
        
        if self.monitor_volts:
            tensor_stats(self.w, "Latent Weights:")
            tensor_stats(g, "Soft Binarized Weights:")
        
        # Recurrent Forward Layer
        feedback = self.PREPROCESS(torch.zeros((img.shape[0],self.feedback))).to(img.device)
        
        for r_pass in range(self.r_passes):
            x = torch.cat((feedback, img[:, r_pass * self.data_in:(r_pass + 1) * self.data_in]), dim=1)
            x = F.linear(x, g, bias=None)
            x = self.INV_AMP(x)
            if self.monitor_volts: tensor_stats(x,f"Voltages in Recurrent Stage after pass {r_pass}")
            feedback = x[:,-self.feedback:]

        x = self.dropout(x)
        
        # Hidden Layers as Repeated Passes through the same Crossbar
        for h_pass in range(self.h_layers):
            x = F.linear(x, g, bias=None)
            x = self.INV_AMP(x)
            if self.monitor_volts: tensor_stats(x,f"Voltages after h_layer {h_pass}")
            x = self.dropout(x)

        # Output Classes
        return x[:,:self.output_size]
        
    def backprop(self, ext_lr):
        with torch.no_grad():
            if self.w.grad is not None:
                grad = self.w.grad
                if self.int_norm: grad = self.int_lr*grad/torch.norm(grad)
                if self.monitor_grad: tensor_stats(grad, "Gradients")
                self.w -= ext_lr*grad
                self.w.grad.zero_()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
class AugmentedDataset(Dataset):
    def __init__(self, images, labels, noise_std=0.1):
        self.images = images
        self.labels = labels
        self.noise_std = noise_std

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        noisy_image = image + torch.randn_like(image) * self.noise_std
        return noisy_image, label

## Model Parameters

In [9]:
params_RRAM = {
    "G_ON": 5.5e-5,          # High conductance state
    "G_OFF": 2.88e-6,        # Low conductance state
    "V_INV": 0.6,            # Inverter voltage
    "R_INV": 2e+3,         # Inverter resistance
    "V_1": 0.1,              # High input voltage
    "V_0": -0.1,             # Low input voltage
    "zeta": 1000,               # Sharpness parameter for soft binarization 
    "initial_factor": 0.001,   # Initial weight scaling factor
    "crossbar": (64, 64),      # Crossbar size (rows, columns)
    "input_size": 784,        # Input size
    "recurrence_size": 8,    # Feedback size for recurrent connections
    "output_size": 10,        # Output size
    "hidden_layers": 1,      # Number of hidden layers 
    "monitor_volts": False,        # check post inverter voltages flag
    "monitor_grad": False,        # check gradient values
    "dropout": 0.1,          # Dropout probability  
    "lr": 1,            # Training LR
    "epochs": 100,          # Training Epochs
    "int_lr": 1,           # Internal LR
    "int_norm": True         # Normalizing Gradient
}

model_params = {k: v for k, v in params_RRAM.items() if k not in ["noise_std", "batch_size", "lr", "epochs"]}

model_RRAM = SoftBinaryRecurrentForwardNetwork(**model_params).to(device)
summary(model_RRAM)

Layer (type:depth-idx)                   Param #
SoftBinaryRecurrentForwardNetwork        4,096
├─Dropout: 1-1                           --
Total params: 4,096
Trainable params: 4,096
Non-trainable params: 0

### Loading Past Best Model

In [10]:
try:
    with open(f"Best_Val_Accuracy_h{model_params['hidden_layers']}.txt", 'r') as f: val_best = float(f.read())
    with open(f"Best_Params_h{model_params['hidden_layers']}.txt", 'r') as f: params_best = ast.literal_eval(f.read())

    model_best = SoftBinaryRecurrentForwardNetwork(**model_params).to(device)

    print("Accuracy:", val_best)
    print("Parameters:", params_best)

    checkpoint = torch.load(f"Best_model_h{model_params['hidden_layers']}.pth")
    model_best.load_state_dict(checkpoint)
    
except Exception as e:
    print(e)
    val_best = 0
    print("No Saved Model")

Accuracy: 19.67
Parameters: {'G_ON': 5.5e-05, 'G_OFF': 2.88e-06, 'V_INV': 0.6, 'R_INV': 2000.0, 'V_1': 0.1, 'V_0': -0.1, 'zeta': 100, 'initial_factor': 0.001, 'crossbar': (64, 64), 'input_size': 784, 'recurrence_size': 36, 'output_size': 10, 'hidden_layers': 1, 'monitor_volts': False, 'monitor_grad': True, 'dropout': 0.3, 'lr': 1, 'epochs': 15, 'int_lr': 1, 'int_norm': True}


## Training:

In [11]:
history_RRAM = {
    "train_loss": [],
    "train_accuracy": [],
    "val_loss": [],
    "val_accuracy": [],
    "gradients":[]
}
criterion = nn.CrossEntropyLoss()

In [13]:
lr = params_RRAM["lr"]
num_epochs = params_RRAM["epochs"]

scaler = GradScaler()

optimizer = torch.optim.Adam(model_RRAM.parameters(), lr=lr)

for epoch in range(num_epochs):
    if epoch == 0:
        lr /= 25
    elif epoch <= 2:
        lr *= 5

    model_RRAM.train()
    train_loss = 0
    train_accuracy = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with autocast():
            outputs = model_RRAM(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.detach() * inputs.size(0)
        _, predictions = torch.max(outputs, dim=1)
        train_accuracy += (predictions == labels).sum().item()
        total_samples += inputs.size(0)

    train_loss /= total_samples
    train_accuracy = (train_accuracy / total_samples) * 100

    model_RRAM.eval()
    total_val_samples = 0
    val_loss = 0
    val_correct = 0

    with torch.no_grad():
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device, non_blocking=True), val_labels.to(device, non_blocking=True)
            with autocast():
                val_outputs = model_RRAM(val_inputs)
                val_loss += criterion(val_outputs, val_labels).item() * val_inputs.size(0)
                _, val_predictions = torch.max(val_outputs, dim=1)
                val_correct += (val_predictions == val_labels).sum().item()
                total_val_samples += val_inputs.size(0)

    val_loss /= total_val_samples
    val_accuracy = (val_correct / total_val_samples) * 100

    history_RRAM["train_loss"].append(train_loss)
    history_RRAM["train_accuracy"].append(train_accuracy)
    history_RRAM["val_loss"].append(val_loss)
    history_RRAM["val_accuracy"].append(val_accuracy)

    if val_accuracy > val_best:
        val_best = val_accuracy
        torch.save(model_RRAM.state_dict(), f"Best_model_h{model_RRAM.h_layers}.pth")
        with open(f"Best_Val_Accuracy_h{model_RRAM.h_layers}.txt", "w") as f:
            f.write(f"{val_best:.6f}")
        with open(f"Best_Params_h{model_RRAM.h_layers}.txt", "w") as f:
            f.write(f"{params_RRAM}")
        print(f"Model saved with Validation Accuracy: {val_best:.6f}")

    print(f"Epoch {epoch + 1}, LR: {lr:.4f}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    if (epoch + 1) % ((num_epochs + 1) // 10) == 0 and epoch != 0:
        lr /= 3

Epoch 1, LR: 0.0400, Train Loss: 2.3221, Train Accuracy: 9.22%, Val Loss: 2.3214, Val Accuracy: 8.92%
Epoch 2, LR: 0.2000, Train Loss: 2.3228, Train Accuracy: 9.19%, Val Loss: 2.3214, Val Accuracy: 8.92%


KeyboardInterrupt: 

In [ ]:
plot_history(history_RRAM, num_epochs, "RRAM")

## Testing

### Current Model

In [ ]:
print(0 + (model_RRAM.w > 0))

In [ ]:
cm = test(model_RRAM, val_inputs, val_labels, class_names = ["A", "T", "V", "X"])

## Best Model:

In [ ]:
0+1*(model_best.w>0)

In [ ]:
cm = test(model_best, val_inputs, val_labels, class_names = ["A", "T", "V", "X"])

## PWL Generation

Let's assume that we will program the two crossbars with seperate PWLs. That is, during programming, we will cut the Inverting Amplifier stages with a pass transistor and connect the programming lines with a pass transistor. First array has 16 Top PWLs and 8 Bottom PWLs. Second array has 8 Top PWLs and 4 Bottom PWLs. And then once the programming switch is toggled to inference mode, only the 16 Top PWLs are to be changed. Let's also generate a PWL for that too.

In the code below, we will first maintain tuples for each PWL that holds what the voltage should be. And then we will write a function that will take there and space pulses of the given voltage that are 100us apart from other and have an ON duration of 100us 

In [ ]:
WL_FC1 = [list() for i in range(16)]
BL_FC1 = [list() for i in range(8)]
WL_FC2 = [list() for i in range(8)]
BL_FC2 = [list() for i in range(4)]
Mode = []
Mode_B = []

V_WRITE = 1.5
V_READ = 0.1
V_mode = 1.2

#### Fully Connected Weights 1

In [ ]:
target = (model_RRAM_best.w1>0).int()
target

In [ ]:
for ind_i, i in enumerate(target):
    for ind_j, j in enumerate(i):
        if j==1: WL_FC1[ind_j].append(V_WRITE)
        else: WL_FC1[ind_j].append(V_WRITE/3)
    for ind_k in range(len(target)): 
        if ind_k==ind_i: BL_FC1[ind_i].append(0)
        else: BL_FC1[ind_k].append(2*V_WRITE/3)

#### Fully Connected Weights 2

In [ ]:
target = (model_RRAM_best.w2>0).int()
target

In [ ]:
for ind_i, i in enumerate(target):
    for ind_j, j in enumerate(i):
        if j==1: WL_FC2[ind_j].append(V_WRITE)
        else: WL_FC2[ind_j].append(V_WRITE/3)
    for ind_k in range(len(target)): 
        if ind_k==ind_i: BL_FC2[ind_i].append(0)
        else: BL_FC2[ind_k].append(2*V_WRITE/3)

#### Filling Out Programming Mode

In [ ]:
WL_FC1 = [i + [0,0] for i in WL_FC1]
BL_FC1 = [i + [0,0] for i in BL_FC1]
while(len(WL_FC2[0]) < len(WL_FC1[0])):
    WL_FC2 = [i + [0,] for i in WL_FC2]
    BL_FC2 = [i + [0,] for i in BL_FC2]
Mode.extend([V_mode]*(len(WL_FC1[0])-1) + [-V_mode])
Mode_B.extend([-V_mode]*(len(WL_FC1[0])-1) + [V_mode])

In [ ]:
print(WL_FC1[0])
print(BL_FC1[0])
print(WL_FC2[0])
print(BL_FC2[0])
print(Mode)
print(Mode_B)
print(len(WL_FC1[0]), len(BL_FC1[0]), len(WL_FC2[0]), len(BL_FC2[0]), len(Mode), len(Mode_B)) 

### Inference: Loading the Testing Set

In [ ]:
val_inputs[0]

In [ ]:
V_1 = 0.1
V_0 = -0.1
include_testing = True
include_every = 4

In [ ]:
if include_testing:
    for i in val_inputs[::include_every]:
        i = i.flatten()
        for ind, j in enumerate(i):
            WL_FC1[ind].append(V_1 if j==1 else V_0)
        BL_FC1 = [i + [0,] for i in BL_FC1]
        WL_FC2 = [i + [0,] for i in WL_FC2]
        BL_FC2 = [i + [0,] for i in BL_FC2]
        Mode = Mode + [-V_mode,]
        Mode_B = Mode_B + [V_mode,]

### PWL Convertion

In [ ]:
def pwl(l):
    t = 0
    res = "pwl(time, 0us, 0V"
    for i in l:
        res += f", {t+5}us, {i:.2f}V, {t+100}us, {i:.2f}V, {t+105}us, 0V, {t+200}us, 0V"
        t+=200
    res += ")"
    return res

In [ ]:
pwl_data = []

for ind, i in enumerate(WL_FC1):
    pwl_data.append({"Signal": f"WL_FC1_{ind}", "Index": ind, "PWL": pwl(i)})
for ind, i in enumerate(BL_FC1):
    pwl_data.append({"Signal": f"BL_FC1_{ind}", "Index": ind, "PWL": pwl(i)})
for ind, i in enumerate(WL_FC2):
    pwl_data.append({"Signal": f"WL_FC2_{ind}", "Index": ind, "PWL": pwl(i)})
for ind, i in enumerate(BL_FC2):
    pwl_data.append({"Signal": f"BL_FC2_{ind}", "Index": ind, "PWL": pwl(i)})
pwl_data.append({"Signal": "Mode", "Index": "", "PWL": pwl(Mode)})
pwl_data.append({"Signal": "Mode_b", "Index": "", "PWL": pwl(Mode_B)})

pwl_data = pd.DataFrame(pwl_data)
pwl_data.to_csv("pwl_data.csv", index=False)
pwl_data.head()

#### Testing Accuracy on 160 Images
ADS isn't allowing PWLs longer than 160 Images, so let's check software accuracy for the same too

In [ ]:
test(model_RRAM_best, val_inputs[::4], val_labels[::4])

In [ ]:
test(model_RRAM_best, train_inputs, train_labels)

## Simulation Data from ADS

In [ ]:
simu = pd.read_csv("Testing_160_Images.csv")
simu

In [ ]:
def remove_units(value):
    return float(value.replace('E', 'e').split('V')[0].replace('sec', ''))

simu['time'] = simu['time'].apply(remove_units)
for col in ['A', 'X', 'V', 'T']:
    simu[col] = simu[col].apply(remove_units)
simu

We just need one sample every 0.1ms samples of these starting from 2.050ms to 33.850ms

In [ ]:
t_stamps = np.arange(2.05e-3, 33.9e-3, 0.2e-3)
t_stamps.shape

In [ ]:
sampled = []
window = 0.02e-3

for t in t_stamps:
    filtered = simu[(simu['time'] >= t - window) & (simu['time'] <= t + window)]
    
    avg_A = filtered['A'].mean()
    avg_X = filtered['X'].mean()
    avg_V = filtered['V'].mean()
    avg_T = filtered['T'].mean()
    
    sampled.append({
        'Image Index': t,
        'A': avg_A,
        'X': avg_X,
        'V': avg_V,
        'T': avg_T
    })

sampled = pd.DataFrame(sampled)
sampled

In [ ]:
def get_max_column(row):
    return row[['A', 'X', 'V', 'T']].idxmax()
sampled['Predicted Class'] = sampled.apply(get_max_column, axis=1)
sampled.to_csv("Sampled_Results.csv", index=False)
sampled

In [ ]:
ground_truth = ['A']*40 + ['X']*40 + ['V']*40 + ['T']*40
correct_predictions = sampled['Predicted Class'] == ground_truth
accuracy = correct_predictions.sum() / len(ground_truth)
print(accuracy*100,end="%")

In [ ]:
plt.figure(figsize=(7, 3.5))

plt.scatter(sampled.index, sampled['A'], color='red', label='A_pred', s=30, marker='o')  # Red dots for A
plt.scatter(sampled.index, sampled['X'], color='blue', label='X_pred', s=30, marker='o')  # Blue dots for X
plt.scatter(sampled.index, sampled['T'], color='green', label='T_pred', s=30, marker='o')  # Green dots for T
plt.scatter(sampled.index, sampled['V'], color='orange', label='V_pred', s=30, marker='o')  # Orange dots for V

plt.xlabel('Image Index')
plt.ylabel('Predicted Voltages (V)')
plt.legend()

plt.axvline(x=40, color='gray', linestyle='--', linewidth=2)
plt.axvline(x=80, color='gray', linestyle='--', linewidth=2)
plt.axvline(x=120, color='gray', linestyle='--', linewidth=2)

plt.text(20, plt.ylim()[1]*(-0.8), 'A', fontsize=15, color='black', ha='center')
plt.text(60, plt.ylim()[1]*0.8, 'X', fontsize=15, color='black', ha='center')
plt.text(100, plt.ylim()[1]*0.8, 'V', fontsize=15, color='black', ha='center')
plt.text(140, plt.ylim()[1]*(-0.8), 'T', fontsize=15, color='black', ha='center')

plt.show()

In [ ]:
for epoch in range(num_epochs):
    model_RRAM.train()

    train_loss = 0
    train_accuracy = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_RRAM(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        model_RRAM.backprop(lr)

        train_loss += loss.item() * inputs.size(0)
        _, predictions = torch.max(outputs, dim=1)
        train_accuracy += (predictions == labels).sum().item()
        total_samples += inputs.size(0)

    train_loss /= total_samples
    train_accuracy = (train_accuracy / total_samples) * 100

In [ ]:
def tensor_stats(tensor, name="Tensor"):
    mean_magnitude = tensor.abs().mean().item()
    print(f"{name} - Mean Magnitude: {mean_magnitude:.2e}, Max: {tensor.max().item():.2e}, Min: {tensor.min().item():.2e}")
    
class RecurrentForwardNetwork(nn.Module):
    def __init__(self, G_ON, G_OFF, V_INV, R_INV, V_1, V_0, zeta, initial_factor, 
                 crossbar=(8,8), input_size=16, recurrence_size=4, output_size=4, 
                 hidden_layers=2, monitor_volts=False, monitor_grad=True, dropout=0.2):
        super(RecurrentForwardNetwork, self).__init__()

        # Crossbar Weights
        self.w = nn.Parameter(torch.empty(crossbar, crossbar))
        nn.init.xavier_uniform_(self.w)
        self.w.data = initial_factor * self.w
        
        # Circuit Parameters
        self.G_ON = G_ON
        self.G_OFF = G_OFF
        self.V_INV = V_INV
        self.R_INV = R_INV
        self.V_1 = V_1
        self.V_0 = V_0

        # Architecture Parameters
        self.feedback = recurrence_size
        self.data_in = crossbar[0] - recurrence_size
        self.output_size = output_size
        self.crossbar_in = crossbar[0]
        self.crossbar_out = crossbar[1]
        self.h_layers = hidden_layers
        self.r_passes = input_size // self.data_in
        
        # Training Variables
        self.zeta = zeta
        self.monitor_volts = monitor_volts
        self.monitor_grad = monitor_grad
        self.dropout = nn.Dropout(p=dropout)

    def INV_AMP(self, x):
        return -self.V_INV * torch.tanh(self.R_INV * x / self.V_INV)

    def SOFT_BIN(self, x):
        return ((self.G_ON - self.G_OFF) * torch.sigmoid(x * self.zeta) + self.G_OFF)

    def PREPROCESS(self, img):
        return (self.V_1 - self.V_0) * img.view(img.size(0), -1) + self.V_0

    def forward(self, img):
        # Preprocessing: Two States of input (V_ON and V_OFF)
        img = self.PREPROCESS(img)

        # RRAM Soft Binarization
        g = self.SOFT_BIN(self.w).to(img.device)
        
        if self.monitor_volts:
            print("Latent Weights:", self.w)
            print("Soft Binarized Weights:", g)
        
        # Recurrent Forward Layer
        feedback = self.PREPROCESS(torch.ones((self.feedback,)).to(img.device))
        
        for r_pass in range(self.r_passes):
            x = torch.cat((feedback, img[r_pass * self.data_in:(r_pass + 1) * self.data_in]), dim=0)
            x = F.linear(x, g, bias=False)
            x = self.INV_AMP(x)
            if self.monitor_volts: print(f"Voltages in Recurrent Stage after pass {r_pass}: {x}")
            feedback = x[-self.feedback:]

        x = self.dropout(x)
        
        # Hidden Layers
        for h_pass in range(self.h_layers):
            x = F.linear(x, g, bias=False)
            x = self.INV_AMP(x)
            x = self.dropout(x)  # Apply dropout within the hidden layers
            if self.monitor_volts: print(f"Voltages after h_layer {h_pass}: {x}")

        # Output Classes
        return x[:self.output_size]
        
    def backprop(self, lr):
        if self.monitor_volts:
            print("Latent Gradients:", lr * self.w.grad if self.w.grad is not None else None)
    
        with torch.no_grad():
            if self.w.grad is not None:
                self.w -= lr * self.w.grad
                self.w.grad.zero_()